In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim


In [7]:
tensor = torch.randn(3, 4, 2).permute(0, 2, 1)
tensor, tensor.shape, tensor.dtype, tensor.device

(tensor([[[-1.0086, -0.1969,  0.4743,  1.2332],
          [ 0.8735,  0.8753,  0.0528,  0.9843]],
 
         [[ 0.6155, -0.3092,  0.7226,  0.2468],
          [ 0.8574, -0.4524,  2.1020,  0.0915]],
 
         [[-0.4897,  0.7155, -1.6454,  0.0726],
          [-0.0483, -0.2290,  0.2160, -2.1783]]]),
 torch.Size([3, 2, 4]),
 torch.float32,
 device(type='cpu'))

In [8]:
mask = tensor > 0
mask, mask.shape, mask.dtype, mask.device

(tensor([[[False, False,  True,  True],
          [ True,  True,  True,  True]],
 
         [[ True, False,  True,  True],
          [ True, False,  True,  True]],
 
         [[False,  True, False,  True],
          [False, False,  True, False]]]),
 torch.Size([3, 2, 4]),
 torch.bool,
 device(type='cpu'))

In [9]:
valid_mask = mask.any(dim=0).any(dim=1)
valid_mask_index = torch.where(valid_mask)[0]
valid_mask_index

tensor([0, 1])

In [11]:
tensor[mask]

tensor([0.4743, 1.2332, 0.8735, 0.8753, 0.0528, 0.9843, 0.6155, 0.7226, 0.2468,
        0.8574, 2.1020, 0.0915, 0.7155, 0.0726, 0.2160])

In [10]:
tensor[valid_mask_index]

tensor([[[-1.0086, -0.1969,  0.4743,  1.2332],
         [ 0.8735,  0.8753,  0.0528,  0.9843]],

        [[ 0.6155, -0.3092,  0.7226,  0.2468],
         [ 0.8574, -0.4524,  2.1020,  0.0915]]])